# Notebook MongoDB : Import des données et requêtes

Ce notebook montre comment importer des fichiers JSON dans MongoDB et effectuer 20 requêtes classiques.

In [ ]:
# Imports
import os
import json
from pymongo import MongoClient
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv()

# Définir les chemins
BASE_DIR = os.path.dirname(os.path.abspath(""))
DATA_DIR = os.path.join(BASE_DIR, "data")  # Assure-toi que les fichiers JSON sont dans ce dossier

# Connexion MongoDB
MONGO_MODE = os.getenv("MONGO_MODE", "local")
MONGO_URI = os.getenv("MONGO_ATLAS_URI") if MONGO_MODE == "atlas" else os.getenv("MONGO_LOCAL_URI")
DB_NAME = os.getenv("DB_NAME", "rh_management")
client = MongoClient(MONGO_URI)
db = client[DB_NAME]

### Fonction pour insérer un fichier JSON dans une collection

In [ ]:
def insert_json(filename, collection_name):
    filepath = os.path.join(DATA_DIR, filename)
    if not os.path.exists(filepath):
        print(f"❌ Fichier non trouvé : {filepath}")
        return
    
    with open(filepath, encoding='utf-8') as f:
        data = json.load(f)
        if isinstance(data, list):
            result = db[collection_name].insert_many(data)
            print(f"✅ Insertion dans {collection_name} : {len(result.inserted_ids)} documents.")
        else:
            print(f"⚠️ Format invalide dans {filename} (doit être une liste JSON).")

### Import des collections

In [ ]:
# Insérer les données JSON dans les collections
insert_json('departements.json', 'departements')
insert_json('postes.json', 'postes')
insert_json('employes.json', 'employes')

# Liste des 20 requêtes MongoDB sur les collections

Chaque requête est suivie d'une explication.

In [ ]:
# 1. Compter le nombre de documents dans chaque collection
print("Nombre de documents dans chaque collection :")
for coll_name in ['departements', 'postes', 'employes']:
    count = db[coll_name].count_documents({})
    print(f"- {coll_name} : {count} documents")

Cette requête compte tous les documents d’une collection.

In [ ]:
# 2. Trouver tous les départements dont le nom contient 'Tech'
results = db.departements.find({"nom": {"$regex": "Tech", "$options": "i"}})
print("Départements avec 'Tech' dans le nom :")
for doc in results:
    print(doc)

Recherche par expression régulière insensible à la casse sur le champ `nom`.

In [ ]:
# 3. Lister tous les postes avec un salaire max supérieur à 500000
results = db.postes.find({"salaire_max": {"$gt": 500000}})
print("Postes avec salaire max > 500000 :")
for doc in results:
    print(doc)

Filtrer les postes selon le salaire maximum.

In [ ]:
# 4. Trouver un employé par son prénom 'Alice'
doc = db.employes.find_one({"prenom": "Alice"})
print("Employé avec prénom Alice :", doc)

Recherche d’un document unique avec `find_one`.

In [ ]:
# 5. Mettre à jour le salaire min des postes 'ingénieur' à 400000
result = db.postes.update_many(
    {"intitule": {"$regex": "ingénieur", "$options": "i"}},
    {"$set": {"salaire_min": 400000}}
)
print(f"Postes mis à jour : {result.modified_count}")

Met à jour plusieurs documents dont le titre contient 'ingénieur'.

In [ ]:
# 6. Supprimer les employés avec un salaire inférieur à 200000
result = db.employes.delete_many({"salaire": {"$lt": 200000}})
print(f"Employés supprimés : {result.deleted_count}")

Supprime les documents selon une condition.

In [ ]:
# 7. Trouver les employés travaillant dans le département 'IT'
results = db.employes.find({"departement": "IT"})
print("Employés dans le département IT :")
for doc in results:
    print(doc)

Filtrer sur un champ exact.

In [ ]:
# 8. Afficher les postes triés par salaire max décroissant
results = db.postes.find().sort("salaire_max", -1)
print("Postes triés par salaire max décroissant :")
for doc in results:
    print(doc)

Tri des résultats.

In [ ]:
# 9. Compter le nombre d'employés par département (agrégation)
pipeline = [
    {"$group": {"_id": "$departement", "total_employes": {"$sum": 1}}}
]
results = db.employes.aggregate(pipeline)
print("Nombre d'employés par département :")
for doc in results:
    print(doc)

Utilisation d'un pipeline d'agrégation pour compter par groupe.

In [ ]:
# 10. Trouver les postes dont le salaire min est entre 300000 et 500000
results = db.postes.find({"salaire_min": {"$gte": 300000, "$lte": 500000}})
print("Postes avec salaire min entre 300000 et 500000 :")
for doc in results:
    print(doc)

Filtrer avec une condition entre deux valeurs.

In [ ]:
# 11. Mettre à jour tous les employés pour ajouter un champ 'actif' à True
result = db.employes.update_many({}, {"$set": {"actif": True}})
print(f"Employés mis à jour (ajout champ 'actif'): {result.modified_count}")

Ajout d'un champ booléen à tous les documents.

In [ ]:
# 12. Trouver les employés dont le prénom commence par 'J'
results = db.employes.find({"prenom": {"$regex": "^J", "$options": "i"}})
print("Employés dont le prénom commence par 'J' :")
for doc in results:
    print(doc)

Recherche par expression régulière sur le début du prénom.

In [ ]:
# 13. Supprimer les départements sans employés (supposons champ 'employes_count')
result = db.departements.delete_many({"employes_count": 0})
print(f"Départements supprimés (sans employés): {result.deleted_count}")

Suppression conditionnelle.

In [ ]:
# 14. Trouver les postes dont la description contient 'gestion'
results = db.postes.find({"description": {"$regex": "gestion", "$options": "i"}})
print("Postes avec 'gestion' dans la description :")
for doc in results:
    print(doc)

Recherche par texte dans une description.

In [ ]:
# 15. Afficher les 5 premiers employés triés par salaire décroissant
results = db.employes.find().sort("salaire", -1).limit(5)
print("Top 5 employés par salaire :")
for doc in results:
    print(doc)

Limite le nombre de résultats et trie.

In [ ]:
# 16. Compter les postes dont le salaire min est supérieur à 350000
count = db.postes.count_documents({"salaire_min": {"$gt": 350000}})
print(f"Nombre de postes avec salaire min > 350000 : {count}")

Comptage avec condition.

In [ ]:
# 17. Mettre à jour tous les départements pour ajouter un champ 'actif' à True
result = db.departements.update_many({}, {"$set": {"actif": True}})
print(f"Départements mis à jour (ajout champ 'actif'): {result.modified_count}")

Ajout d'un champ booléen.

In [ ]:
# 18. Trouver les employés qui ont un poste de 'manager'
results = db.employes.find({"poste": {"$regex": "manager", "$options": "i"}})
print("Employés avec un poste de manager :")
for doc in results:
    print(doc)

Recherche avec expression régulière.

In [ ]:
# 19. Supprimer tous les employés inactifs (champ 'actif' à False)
result = db.employes.delete_many({"actif": False})
print(f"Employés inactifs supprimés : {result.deleted_count}")

Suppression conditionnelle.

In [ ]:
# 20. Trouver les postes dont le champ 'description' existe
results = db.postes.find({"description": {"$exists": True}})
print("Postes avec champ 'description' :")
for doc in results:
    print(doc)